In [1]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split, feature_dataload
from feature_engineering import *
from candidate import *
from tqdm import tqdm

../../src/dataload.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def day_feature(df_train:pd.DataFrame())->pd.Series():


# dataload
- 편의상 df_train_week 변수 통일하여 사용함

In [2]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [3]:
# # # 전체 데이터 이용시
# df_history = pd.read_csv(path+'history_data.csv')

# # ## 날짜 전처리
# df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
# df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
# df_history = df_history.drop("log_time", axis=1)

# df_train_week = df_history.copy()

# # # month 이용
# # # df_train_week = df_train_month.copy()

# feature dataload

In [4]:
# data load
watch_df, buy_df, search_df, meta_df, profile_df = feature_dataload(path)

../../src/dataload.py:87: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path + "meta_data.csv")


In [5]:
# feature engineering
# df_train_week = history_feature_engineering(df_train_week)

# short_con_freq_feature = df_train_week[["album_id","short_trailer","continuous_play","album_viewcount_freq"]]

watch_df_feature = watch_feature_engineering(watch_df)
watch_df_feature = watch_df_feature.drop_duplicates(subset=["profile_id","album_id"])

paid_df_feature = paid_feature_engineering(df_train_week, buy_df)

searched_df_feature = searched_feature_engineering(df_train_week, search_df)

meta_df_feature = meta_feature_engineering(meta_df)

profile_df_feature = profile_feature_engineering(profile_df)
profile_df_feature = profile_df_feature[["profile_id","sex","age","age_bin"]]

# day_week feature engineering
- to split label, train data

In [6]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-23 00:00:00
min day: 0 max day: 53
min week: 0 max week: 7


# label & train split

In [7]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week, cv_date=0)

split last week: 7


In [8]:
n = 25

In [9]:
len(df_train), len(df_train_week)

(945518, 945518)

# prerpocess

In [10]:
# 총 3개 파일 사용

# log dt 기준 duplicates
df_train_week = df_train.drop_duplicates(subset=["profile_id","album_id","log_dt"])
# ss_id 기준 duplicates
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
# album_id, profile_id 기준으로 duplicates
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

# 전체 유저 한정
customers = df_train.profile_id.unique()
# 라벨 한정
# customers = label_df.profile_id.unique()

# fav_cast feature

In [11]:
favorite_cast_feature = fav_cast(df_train, meta_df_feature)

# fav_keyword_feature

In [12]:
meta_plus_df = pd.read_csv(path+"meta_data_plus.csv")
fav_keyword_feature = fav_keyword(df_train, meta_plus_df)

# day week features

In [13]:
interaction_day_week_first = day_week_feature(df_train_week)
hour_feature = hour_feature(df_train_week)

../../src/feature_engineering.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)


# paid_count_feature

In [14]:
paid_count_feature = paid_count_feature(df_train_week)

# candidate generation

# MP

In [15]:
MP_cand, MP_feature = most_popular(df_train)

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [16]:
%%time
general_MP_cand, general_MP_feature = general_most_popular(df_train)

CPU times: user 41.8 ms, sys: 11.1 ms, total: 52.8 ms
Wall time: 51.6 ms


# personal_MP

In [49]:
%%time
personal_MP_candidate, personal_MP_feature = personal_most_popular(personal_train, N=50)

100%|█████████████████████████████████████| 8057/8057 [00:07<00:00, 1124.40it/s]


CPU times: user 7.53 s, sys: 146 ms, total: 7.68 s
Wall time: 7.67 s


# MP_user_genre

In [18]:
# 이전에는 meta 중복 때문에 장르 count가 잘못 뻥튀기 되어서 잘못 MP가 매겨짐
# feature merge 시 genre_mid 와 profile_id 기준으로 해야함
genre_candidate, user_genre_feature = user_genre_most_popular(personal_train, meta_df)

# ALS MF candidate

In [19]:
%%time
from ALS_MF import MF
model = MF(df_train_week, clf={'factors': 200, 'regularization': 0.005, 'iterations': 3, 'n': 200})

user_factors_feature = pd.read_parquet(path+"profile_factor_als.parquet")
item_factors_feature = pd.read_parquet(path+"album_factor_als.parquet")

als_candidate, _, _, als_model = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20383, 200) user vector shape (8057, 200)
CPU times: user 21.2 s, sys: 18.5 s, total: 39.7 s
Wall time: 22.5 s


# apriori candidaate

In [20]:
# %%time
# from apriori import apriori_train, apriori_candidate
# rules_confidence_item_week = apriori_train(df_train, 0.1, 0.8)
# apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

# NCF candidate

In [21]:
ncf_candidate_df = pd.read_csv(path + "2_neg100_submission.csv")

In [22]:
import ast

ncf_candidate_list = []
for idx, row in ncf_candidate_df.iterrows():
    ncf_df = pd.DataFrame()
    ncf_df["album_id"] = ast.literal_eval(row.predicted_list)
    ncf_df["profile_id"] = row.profile_id
    ncf_candidate_list.append(ncf_df)
ncf_candidate = pd.concat(ncf_candidate_list)
ncf_candidate = ncf_candidate[ncf_candidate["profile_id"].isin(set(df_train.profile_id))]

# 아웃라이어 삭제
album_max_dict = df_train.groupby("profile_id")["album_id"].max().to_dict()

last_train_df_list = []
for user_id in ncf_candidate.profile_id.unique():
    last_train_df_list.append(ncf_candidate[(ncf_candidate["profile_id"]==user_id)&(ncf_candidate["album_id"]<=album_max_dict[user_id])])
ncf_candidate = pd.concat(last_train_df_list)

# series candidate

In [23]:
series_candidate = series_candidate(df_train_week, threshold=30)

Train shape: (156060, 13)


1897it [00:07, 257.10it/s]
100%|██████████████████████████████████████| 8057/8057 [00:11<00:00, 691.07it/s]


6867


# age candidate

In [24]:
# age_MP_candidate = age_MP(df_train_week, profile_df_feature)

# candidate merge

In [25]:
# 평균 candidate 수
MP_cand.groupby("profile_id").count().mean()[0],\
personal_MP_candidate.groupby("profile_id").count().mean()[0],\
genre_candidate.groupby("profile_id").count().mean()[0],\
als_candidate.groupby("profile_id").count().mean()[0],\
ncf_candidate.groupby("profile_id").count().mean()[0],\
series_candidate.groupby("profile_id").count().mean()[0]

(50.0, 50.0, 14.584212486036986, 200.0, 23.43601836912002, 22.526722003786222)

In [50]:
candidate_0 = MP_cand
candidate_1 = general_MP_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
# candidate_5 = apriori_candidate.copy()
candidate_6 = ncf_candidate.copy()
candidate_7 = series_candidate[["profile_id","album_id"]]
# candidate_8 = age_MP_candidate.copy()

candidate = [candidate_0, candidate_1,candidate_2, candidate_3, candidate_4,candidate_6, candidate_7]

cand = pd.concat(candidate)

cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.album_id.nunique())

cand 데이터 수: 1968152
cand nunique:  16896


# features merge

In [51]:
candidate = pd.merge(cand, MP_feature, how='left', on='album_id')
# general counts feature merge
candidate = pd.merge(candidate, general_MP_feature, how='left', on='album_id')
# personal_count feature merge
candidate = pd.merge(candidate, personal_MP_feature, how='left', on=['profile_id','album_id'])
candidate["personal_counts"] = candidate["personal_counts"].fillna(1)
# meta feature
candidate = pd.merge(candidate, meta_df_feature, how="left",on="album_id")
# genre feature merge(genre_cnt, user_genre_total_cnt, user_genre_total_percent)
candidate = pd.merge(candidate, user_genre_feature, how='left', on=['profile_id','genre_mid'])
# apriori feature merge
# candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [52]:
print('cand 피처 수:',len(candidate))

cand 피처 수: 1968152


In [96]:
# meta feature merge

'''
아직 사용 안하고 있음
# profile feature add
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
# meta feature add
candidate_add_features = pd.merge(candidate_add_features, meta_df.drop_duplicates('album_id'), how='left', on='album_id')

# catetegory feature label encoding

columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
'genre_mid','cast_1','cast_2','cast_3']
from sklearn.preprocessing import LabelEncoder
for col in columns:
    LE = LabelEncoder()
    candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

# drop columns

candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
                                                              'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
                                                              ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
                                                              ,'ch_interest_keyword_cd_3',\
                                                                'genre_mid','cast_1','cast_2','cast_3'])
                                                                
'''
candidate_add_features = candidate.copy()

# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

# interaction day, week merge
candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_first, how="left",on=["profile_id","album_id"])

In [97]:
candidate_add_features = pd.merge(candidate_add_features, watch_df_feature, how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, profile_df_feature, how="left",on="profile_id")
candidate_add_features = pd.merge(candidate_add_features, searched_df_feature,how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, paid_df_feature,how="left",on=["profile_id","album_id"])
candidate_add_features["paid_label"] = candidate_add_features["paid_label"].fillna(0)

####### candidate_add_features = pd.merge(candidate_add_features, favorite_cast_feature, how="left",on="profile_id")
candidate_add_features = pd.merge(candidate_add_features, fav_keyword_feature, how="left",on="profile_id")
# candidate_add_features = pd.merge(candidate_add_features, paid_count_feature, how="left",on="profile_id")
# 타입오류 픽처 제거
# candidate_add_features = candidate_add_features.drop(columns=["onair_date","title", "genre_small", "cast_1", "cast_2", "cast_3", "cast_4", "cast_5", "cast_6", "cast_7"])

In [85]:
len(candidate_add_features)

1968152

In [56]:
# df_user_replay_bool_ver1 = pd.read_csv(path+"df_user_replay_bool.csv")
# df_user_replay_bool_ver2 = pd.read_csv(path+"df_user_replay_bool2.csv")
# maping_df = df_train[["profile_id"]]
# maping_df["replay"] = True
# maping_df = maping_df[~(maping_df["profile_id"].isin(df_user_replay_bool_ver2.profile_id))]
# df_user_replay_bool = pd.concat([df_user_replay_bool_ver1,maping_df]).drop_duplicates()
# replay_users = df_user_replay_bool[df_user_replay_bool["replay"]==True].profile_id.unique()

In [66]:
# repurchase_df = df_train[["profile_id","album_id"]].drop_duplicates()
# repurchase_cand = pd.merge(repurchase_df, candidate_add_features, how="left", on=["profile_id","album_id"])

In [86]:
repurchase_dict = df_train[["profile_id","album_id"]].groupby("profile_id")["album_id"].unique().to_dict()

repurchase_df_list = []
for user_id in tqdm(df_train.profile_id.unique()):
    repurchase_df_list.append(candidate_add_features[(candidate_add_features["profile_id"]==user_id)&(candidate_add_features["album_id"].isin(repurchase_dict[user_id]))])
repurchase_df = pd.concat(repurchase_df_list)

100%|███████████████████████████████████████| 8057/8057 [04:02<00:00, 33.16it/s]


In [87]:
repurchase_dict = df_train[["profile_id","album_id"]].groupby("profile_id")["album_id"].unique().to_dict()

non_repurchase_df_list = []
for user_id in tqdm(df_train.profile_id.unique()):
    non_repurchase_df_list.append(candidate_add_features[(candidate_add_features["profile_id"]==user_id)&~(candidate_add_features["album_id"].isin(repurchase_dict[user_id]))])
non_repurchase_df = pd.concat(non_repurchase_df_list)

100%|███████████████████████████████████████| 8057/8057 [03:12<00:00, 41.75it/s]


In [98]:
len(repurchase_df), len(non_repurchase_df)

(293577, 1674575)

In [99]:
# 라벨 부여
def label_create(candidate_add_features_df, label_df):
    train_df = pd.merge(candidate_add_features_df, label_df, how='left', on=['profile_id','album_id'])
    train_df['rating'] = train_df['rating'].fillna(0)
    return train_df

# 비복원 추출 defalut
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# repurchase_train_df = label_create(repurchase_df, label_df)
# non_repurchase_train_df = label_create(non_repurchase_df, label_df)
train_df = label_create(candidate_add_features, label_df)

In [100]:
train_df["rating"].value_counts()

0.0    1917685
1.0      50467
Name: rating, dtype: int64

In [93]:
repurchase_train_df["rating"].value_counts()

0.0    243110
1.0     50467
Name: rating, dtype: int64

In [94]:
non_repurchase_train_df["rating"].value_counts()

0.0    1674575
Name: rating, dtype: int64

# model

In [61]:
# # # feature selection 컬럼만 학습
# train_df = train_df[['profile_id','album_id','day', 'personal_counts', 'week', 'total_counts',
#        'item_vector66', 'user_vector51', 'user_vector119', 'item_vector76',
#        'user_vector59', 'item_vector97', 'user_vector164', 'user_vector196',
#        'user_vector178', 'user_vector147', 'user_vector99', 'user_vector167',
#        'item_vector165', 'user_vector75', 'user_vector50', 'user_vector136',
#        'user_vector108', 'user_vector131', 'user_vector18', 'user_vector49',
#        'user_vector62', 'user_vector152', 'user_vector177', 'item_vector119','rating']]

In [285]:
def lgbmrank_create(train_df):
    lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':45})
    X_train, sample_submission = lgbmrank.valid_evaluation()
    return X_train, sample_submission

In [ ]:
X_train, sample_submission = lgbmrank_create(train_df)
# X_train_1, sample_submission_1 = lgbmrank_create(repurchase_train_df)
# X_train_2, sample_submission_2 = lgbmrank_create(non_repurchase_train_df)

In [ ]:
X_train.pred.value_counts()

In [126]:
'''
n_estimators: 5 -> 300
n_estimators: 10 -> 900
n_estimators: 15 -> 2500
n_estimators: 20 -> 4118
n_estimators: 25 -> 6532
n_estimators: 30 -> 9267
n_estimators: 35 -> 10601
n_estimators: 40 -> 11415
n_estimators: 45 -> 11415
'''

In [127]:
# repu
X_train = X_train[["profile_id","album_id","pred"]]
repurchase_dict = df_train[["profile_id","album_id"]].groupby("profile_id")["album_id"].unique().to_dict()

repurchase_df_list = []
for user_id in tqdm(df_train.profile_id.unique()):
    repurchase_df_list.append(X_train[(X_train["profile_id"]==user_id)&(X_train["album_id"].isin(repurchase_dict[user_id]))])
repurchase_df = pd.concat(repurchase_df_list)

100%|███████████████████████████████████████| 8057/8057 [03:02<00:00, 44.07it/s]


In [128]:
repurchase_dict = df_train[["profile_id","album_id"]].groupby("profile_id")["album_id"].unique().to_dict()

non_repurchase_df_list = []
for user_id in tqdm(df_train.profile_id.unique()):
    non_repurchase_df_list.append(X_train[(X_train["profile_id"]==user_id)&~(X_train["album_id"].isin(repurchase_dict[user_id]))])
non_repurchase_df = pd.concat(non_repurchase_df_list)

100%|███████████████████████████████████████| 8057/8057 [03:15<00:00, 41.23it/s]


In [139]:
repurchase_df = repurchase_df.sort_values(by=["profile_id","pred"], ascending=False)
non_repurchase_df = non_repurchase_df.sort_values(by=["profile_id","pred"], ascending=False)

In [255]:
repurchase_pred_df= repurchase_df.groupby("profile_id")["album_id"].unique().reset_index()
repurchase_pred_df["album_id"] = repurchase_pred_df["album_id"].apply(lambda x: x[:25])

In [256]:
non_repurchase_dict = non_repurchase_df.groupby("profile_id")["album_id"].unique().to_dict()

In [257]:
pred_list = {}
for idx, row in repurchase_pred_df.iterrows():
    pred_list[row.profile_id] = row.album_id.tolist() + non_repurchase_dict.get(row.profile_id,[]).tolist()
    
repurchase_pred_df["album_id"] = repurchase_pred_df["profile_id"].apply(lambda x: pred_list.get(x,[]))

In [258]:
lgbm_user_items_dict = repurchase_pred_df.set_index("profile_id").to_dict()

In [259]:
sample_sumbission_week['album_id'] = sample_sumbission_week['profile_id'].apply(lambda x: (lgbm_user_items_dict["album_id"].get(x, []))[:25])

print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))

lgbm ndcg: 0.20525061938585823


In [265]:
# non_repurchase_df.groupby("profile_id")["album_id"].unique().tail(60)

In [122]:
X_train.pred.value_counts()

-0.049072    1674575
-0.048614      97832
-0.049010      63205
 0.049087      30335
-0.041997       7967
              ...   
 0.003883          1
 0.006629          1
-0.005647          1
 0.013988          1
 0.028930          1
Name: pred, Length: 322, dtype: int64

In [118]:
X_train.pred.value_counts()

-0.077108    1674575
-0.076137      97832
-0.077046      63205
 0.077129      30335
 0.022683       5166
              ...   
 0.025709          1
 0.017070          1
 0.018389          1
 0.048510          1
 0.029454          1
Name: pred, Length: 956, dtype: int64

In [125]:
X_train[["profile_id","album_id","pred"]].groupby("profile_id")

,profile_id,album_id,pred
0,25844,124,-0.049072
1,25844,16,-0.048614
2,25844,125,-0.049072
3,25844,241,-0.049010
4,25844,65,-0.049072
...,...,...,...
1968147,28681,3670,-0.049072
1968148,28681,3696,-0.049072
1968149,19485,41,-0.049072
1968150,19485,43,-0.049072


In [113]:
X_train.groupby("profile_id")

In [95]:
cnt =0
for i in sample_submission.album_id:
    if len(i)<25:
        cnt += 1
print(cnt)

533


In [42]:
label_users = label_df.profile_id.unique()
label_train_df = train_df[train_df["profile_id"].isin(label_users)]

label_train_df["rating"].value_counts()

0.0    216859
1.0     58731
Name: rating, dtype: int64

# 모델 label 유저만 학습

In [43]:
lgbmrank = LGBMRank(label_train_df, mode='week', model_params={'n_estimators':5})
model, feature_importances_df = lgbmrank.train()
# X_train, sample_sumbission = lgbmrank.valid_evaluation()

In [44]:
# 모델 앙상블 모델 파일 만들기
# import pickle

# file = 'lgb_model_ncf_3.pkl'
# pickle.dump(model, open(file, 'wb'))
# print('Trained LGB model was saved!')

# # load model
# import joblib
# lgb_model_1 = joblib.load('lgb_model_ncf_1.pkl')
# lgb_model_2 = joblib.load('lgb_model_ncf_2.pkl')
# lgb_model_3 = joblib.load('lgb_model_ncf_3.pkl')

In [45]:
user_idx = train_df.profile_id
item_idx = train_df.album_id

del train_df["profile_id"], train_df["album_id"], train_df["rating"]

In [46]:
pred = model.predict(train_df)
train_df["profile_id"] = user_idx
train_df["album_id"] = item_idx
train_df["pred"] = pred

lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
sample_sumbission_week['album_id'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))

print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))

lgbm ndcg: 0.19678160505644177


# 최종 제출

In [40]:
sample_submission_week = pd.read_csv(path + 'sample_submission.csv')

In [41]:
pred = model.predict(train_df)
train_df["profile_id"] = user_idx
train_df["album_id"] = item_idx
train_df["pred"] = pred

lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
sample_submission_week['predicted_list'] = sample_submission_week['profile_id'].apply(lambda x: list(lgbm_user_items_dict.get(x, [])))

In [42]:
sample_submission_week.to_csv("submission_1201_ver2.csv",index=False)

# 모델 CV 부분

In [ ]:
# pred_1 = lgb_model_1.predict(train_df)
# pred_2 = lgb_model_2.predict(train_df)
# pred_3 = lgb_model_3.predict(train_df)

# pred = np.mean([pred_1, pred_2, pred_3], axis=0)
# train_df["profile_id"] = user_idx
# train_df["album_id"] = item_idx
# train_df["pred"] = pred

# lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
# lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
# sample_sumbission_week['album_id'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))

# print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))

In [ ]:
def evaluation(model, train_df, df_train, sample_sumbission_week,MP_cand, user_idx, item_idx):
    pred = model.predict(train_df)
    train_df["profile_id"] = user_idx
    train_df["album_id"] = item_idx
    train_df["pred"] = pred

    album_max_dict = df_train.groupby("profile_id")["album_id"].max().to_dict()

    last_train_df_list = []
    for user_id in train_df.profile_id.unique():
        last_train_df_list.append(train_df[(train_df["profile_id"]==user_id)&(train_df["album_id"]<=album_max_dict[user_id])])
    after_train_df = pd.concat(last_train_df_list)

    lgbm_sub_df = after_train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sample_sumbission_week['predicted_list'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))
    
#     MP_list = MP_cand.album_id.unique()
    
#     # cold start user file MP_list top25
#     sumbission_cold = sample_sumbission_week.copy()
#     sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
#                                             .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
#     print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))
#     return sumbission_cold
    return sample_sumbission_week, album_max_dict

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
sample_sumbission_week, album_max_dict = evaluation(model, train_df, df_train, submission, MP_cand, user_idx, item_idx)

for i in sample_sumbission_week.predicted_list:
    if len(i)<25:
        print(len(i))

In [ ]:
train_df.groupby("profile_id").count()["album_id"][train_df.groupby("profile_id").count()["album_id"]<25]

In [ ]:
sample_sumbission_week.to_csv("sumbission_ver12.csv",index=False)

In [ ]:
# meta_df = pd.read_csv(path+"meta_data.csv")
meta_df = meta_df.drop(columns=["genre_large","run_time","onair_date"])

In [ ]:
columns = ["title","sub_title","genre_mid","genre_small","country","cast_1","cast_2","cast_3","cast_4","cast_5","cast_6","cast_7"]
meta_one_hot_df = pd.get_dummies(meta_df, columns = columns)

In [ ]:
series_list = []
for idx, row in tqdm(meta_df.iterrows()):
    row_dict = {}
    row_dict["album_id"] = [row["album_id"]]
    for col in columns:
        if type(row[col]) == str:
            row_dict[row[col]] = [1]
    series_list.append(row_dict)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=lgbm_sub_df)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=df_train)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=label_df)
plt.show()

In [ ]:
real_target_list = []
for idx, row in test_answer_week.iterrows():
    df = pd.DataFrame()
    df["album_id"] = row["album_id"]
    df["profile_id"] = row["profile_id"]
    real_target_list.append(df)
real_target_df = pd.concat(real_target_list)
pred_df = lgbm_sub_df[["profile_id","album_id","pred"]]

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=pred_df)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=real_target_df)
plt.show()

In [ ]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [ ]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [ ]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [ ]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [ ]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

In [ ]:
sumbission_cold

In [ ]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [ ]:
sumbission_cold.to_csv('lgbm_candidate_submission_NCF_candidate_ver10.csv', index=False)